In [26]:
import json
import re

class TextoJson:
    def __init__(self, archivo):
        self.archivo = archivo
        self.data = {
            "informacion_paciente": {},
            "informacion_general": {},
            "diagnostico": {},
            "antecedentes": {},
            "habitos_estilo_vida": {},
            "historial_medico": {},
            "historia_personal": {},
            "comorbilidades": {}
        }

    def preprocesar_texto(self):
        with open(self.archivo, 'r', encoding='utf-8') as file:
            lineas = file.readlines()

        texto_procesado = []
        clave_actual = None
        valor_actual = []

        for linea in lineas:
            linea = linea.strip().lower()
            if not linea:
                continue  # Ignorar líneas vacías

            # Identificar si la línea es una nueva clave
            if re.match(r'^[a-záéíóúñ]+.*?:', linea, re.IGNORECASE):
                # Si ya hay una clave en curso, guardar la clave y su valor
                if clave_actual:
                    texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")
                
                # Actualizar clave y reiniciar el valor
                clave_actual = linea.split(':', 1)[0].strip()
                valor_actual = [linea.split(':', 1)[1].strip()]
            elif clave_actual:
                # Agregar la línea al valor actual
                valor_actual.append(linea)
            else:
                ##este caso es cuando la clave no tiene dos puntos
                texto_procesado.append(linea)

        # Guardar la última clave-valor si quedó algo pendiente
        if clave_actual:
            texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")

        print('\n'.join(texto_procesado))
        return '\n'.join(texto_procesado)

    def procesar(self):
        contenido = self.preprocesar_texto().split('\n')

        # Procesar las líneas del contenido
        for linea in contenido:
            # Capturar información personal
            match_id = re.match(r'^(gacmc)\s*/\s*(.+?)\s*/\s*(\d+\s*años)\s*/\s*(.*)$', linea)
            if match_id:
                self.data["informacion_paciente"]["clave"] = match_id.group(1).strip()
                self.data["informacion_paciente"]["id"] = match_id.group(2).strip()
                self.data["informacion_paciente"]["edad"] = match_id.group(3).strip()
                self.data["informacion_paciente"]["medico_tratante"] = match_id.group(4).strip()
                continue

             # Capturar información general
            origen_flag = False
            residencia_flag = False

            if "originaria" in linea or "origen" in linea:
                origen_flag = True
            if "residente" in linea or "residencia" in linea:
                residencia_flag = True

            if origen_flag and residencia_flag:  # Caso "origen y residencia"
                match = re.search(r':\s*(.+?)\s+y\s+(.+)', linea)
                if match:
                    self.data["informacion_general"]["origen"] = match.group(1).strip()
                    self.data["informacion_general"]["residencia"] = match.group(2).strip()
            elif origen_flag:  # Solo origen
                match = re.search(r'(originaria|origen):?\s*(.+)', linea)
                if match:
                    self.data["informacion_general"]["origen"] = match.group(2).strip()
            elif residencia_flag:  # Solo residencia
                match = re.search(r'(residente|residencia):?\s*(.+)', linea)
                if match:
                    self.data["informacion_general"]["residencia"] = match.group(2).strip()

            # Ocupación
            match_ocupacion = re.search(r'ocupación:?\s*(.+)', linea)
            if match_ocupacion:
                self.data["informacion_general"]["ocupacion"] = match_ocupacion.group(1).strip()

            # Seguridad social
            match_ss = re.search(r'^(seguridad social|ss):?\s*(.+)$', linea)  # Asegurar que la línea completa coincida
            if match_ss:
                self.data["informacion_general"]["ss"] = match_ss.group(2).strip()

            # diagnostico}
            match_diagnostico = re.search(r'^(diagnostico|diagnóstico):?\s*(.+)$', linea)  # Asegurar que la línea completa coincida
            if match_diagnostico:
                self.data["diagnostico"] = match_diagnostico.group(2).strip()

            #antecedentes, cualquier tipo de antecedente como ahf, apf, etc
            match_antecedentes = re.search(r'^(ahf|apf|apnp|app|appnp|apm|apnm|appm|appnm|apmnp|apmnm|appmnp|appmnm|appmnmnp)\s+(\w+):?\s*(.+)$', linea)
            if match_antecedentes:
                tipo = match_antecedentes.group(1) ## tipo de antecedente como ahf, apf, etc
                categoria = match_antecedentes.group(2) ## categoria de antecedente como oncologico, etc
                detalle = [detalle.strip() for detalle in re.split(r'[,/|]', match_antecedentes.group(3))]
                
                if tipo not in self.data["antecedentes"]:
                    self.data["antecedentes"][tipo] = {} ## si no existe la categoria de antecedente se crea

                self.data["antecedentes"][tipo][categoria] = detalle

            #  Procesar historial médico
            match_historial = re.search(r'cirugías:(.+)', linea, re.IGNORECASE)
            if match_historial:
                cirugias = match_historial.group(1)
                matches = re.findall(r'([^(]+)\((\d{4})\)', cirugias)
                for match in matches:
                    procedimientos, year = match
                    procedimientos = re.split(r'\s*[+/]\s*', procedimientos.strip())
                    procedimientos = [proc.strip(' /').strip() for proc in procedimientos if proc.strip()]
                    if year not in self.data["historial_medico"]:
                        self.data["historial_medico"][year] = []
                    self.data["historial_medico"][year].extend(procedimientos)

            # Procesar comorbilidades
            match_comorbilidades = re.search(r'comorbilidades:\s*(.+)', linea, re.IGNORECASE)
            if match_comorbilidades:
                comorbilidades = match_comorbilidades.group(1)
                # Ajustar regex para manejar casos con o sin descripción
                matches = re.findall(
                    r'(\d+)\.\s*([^()]+)\((\d{4})\):?\s*(.*?)(?=\d+\.\s*[^()]+\(\d{4}\)|$)',
                    comorbilidades
                )
                for match in matches:
                    _, nombre, year, descripcion = match  # Separar elementos
                    year = year.strip()
                    nombre = nombre.strip()
                    descripcion = descripcion.strip() if descripcion else None  # Manejar descripción vacía
                    
                    # Inicializar el año en el diccionario si no existe
                    if year not in self.data["comorbilidades"]:
                        self.data["comorbilidades"][year] = []
                    
                    # Agregar la comorbilidad al año correspondiente
                    self.data["comorbilidades"][year].append({
                        "nombre": nombre,
                        "descripcion": descripcion  # Permitir que la descripción sea None
                    })

        self.imprimir()

            
    def guardar_json(self, salida):
        with open(salida, 'w', encoding='utf-8') as file:
            json.dump(self.data, file, indent=4, ensure_ascii=False)

    def imprimir(self):
        print(json.dumps(self.data, indent=4, ensure_ascii=False))

# Ejemplo de uso
archivo_entrada = "D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\TextoExtraido2.txt"  # Archivo de entrada con el texto
archivo_salida = "D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\resultado.json"  # Archivo JSON de salida

procesador = TextoJson(archivo_entrada)
procesador.procesar()
# procesador.guardar_json(archivo_salida)

gacmc / 84959 / 67 años / dra. martínez
originaria y residente: guanajuato y ciudad de méxico
ocupación: directora de planeación conalep
seguridad social: issste
diagnóstico: cdi de mama derecha ec la triple negativo
ahf oncológicos: tía materna falleció por cáncer de mama / prima materna con cáncer de mama (desconoce edad al diagnóstico)
apf oncológicos: tía materna falleció por cáncer de mama / prima materna con cáncer de mama (desconoce edad al diagnóstico)
tabaco: negado
alcohol: ocasional
drogas: negado - menarca 15 años / fum 21 años -gopocoao - trh: durante 30 años - estado hormonal: postmenopáusica
cirugías: esplenectomía abierta (1976) / tiroidectomía (1988) / pancreatectomia distal + resección de implante gástrico + biopsia en cuña de segmento ll hepático + colecistectomía y maniobra de kocher extendida (2016) / cc + gc (2017)
comorbilidades: 1. linfoma de hodgkin (1975): 3 ciclos qt (ciclofosfamida, vincristina y otro fármaco) + rt por encima del diafragma (imss) 2. cáncer p

<>:167: SyntaxWarning: invalid escape sequence '\D'
<>:168: SyntaxWarning: invalid escape sequence '\D'
<>:167: SyntaxWarning: invalid escape sequence '\D'
<>:168: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JOSUE\AppData\Local\Temp\ipykernel_16048\698163303.py:167: SyntaxWarning: invalid escape sequence '\D'
  archivo_entrada = "D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\TextoExtraido2.txt"  # Archivo de entrada con el texto
C:\Users\JOSUE\AppData\Local\Temp\ipykernel_16048\698163303.py:168: SyntaxWarning: invalid escape sequence '\D'
  archivo_salida = "D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\resultado.json"  # Archivo JSON de salida
